In [1]:
from nltk import word_tokenize
from collections import defaultdict
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import io
import numpy as np
import nltk

In [2]:
df = pd.read_csv('o365Data.csv', dtype=object)

In [3]:
all_subjects = df['EmailSubject'].tolist()

In [4]:
receipt_subjects = [s.lower() for s in all_subjects if isinstance(s,str)]

In [5]:
import mailbox

In [6]:
mbox = mailbox.mbox('emails.mbox')

In [7]:
non_receipt_emails = []
non_receipt_subjects = []
for message in mbox:
    payload = message.get_payload(decode=False)
    non_receipt_subjects.append(message['Subject'])
    if isinstance(payload,list):
        non_receipt_emails.append(payload[0])
    else:
        non_receipt_emails.append(payload) 


In [8]:
def clean_gmail_subjects(subs): 
    subs = [s for s in subs if isinstance(s,str)]
    
    remove_words = ['invoice','utf','utf-8','receipt','amazon','paypal','payment','confirmation','hughes','trip','booking','lance','weston','order','reservation','confirmed','itinerary','statement','folio','bill']

    return[s.lower() for s in subs if not any(word.lower() in s.lower() for word in remove_words)]

In [9]:
non_receipt_subjects = clean_gmail_subjects(non_receipt_subjects)

In [10]:
import random

def get_test_data_labels(zeros_list,ones_list):
    labels = np.zeros(len(zeros_list)).tolist() + np.ones(len(ones_list)).tolist()
    
    all_data = zeros_list + ones_list
    combined = list(zip(all_data, labels))
    random.shuffle(combined)

    all_data[:], labels[:] = zip(*combined)
    return all_data,labels

In [11]:
subjects, labels = get_test_data_labels(non_receipt_subjects,receipt_subjects)

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
X_test = []
y_test = []
predicted = []

def classify(data, labels):
    global X_test,y_test,predicted
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
    vectorizer = CountVectorizer()
#     clf = MultinomialNB()
    clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=7, tol=None)
    pipe = Pipeline([('vect', vectorizer),
                        ('tfidf', TfidfTransformer()),
                        ('clf', clf)
                        ])
                        
    pipe.fit(X_train, y_train)  
    predicted = pipe.predict(X_test)
    
    return np.mean(predicted == y_test), clf, vectorizer,pipe

In [21]:
accuracy, clf, vectorizer,pipe = classify(subjects,labels)
print(accuracy)


0.9723881327719573


In [17]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

        0.0       0.96      1.00      0.98      6916
        1.0       0.99      0.92      0.96      3297

avg / total       0.97      0.97      0.97     10213



In [ ]:
z = [X_test[i] for i in range(len(X_test)) if y_test[i] == 1 and predicted[i] == 0]
z



In [23]:
body = "monthly parking rate increase"


pipe.predict([body])


array([0.])

In [25]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [26]:
show_most_informative_features(vectorizer,clf,1000)

	-2.3226	modeler        		15.8240	receipt        
	-2.3016	before         		12.8195	order          
	-2.2851	dropbox        		11.1234	payment        
	-2.2176	history        		9.6832	confirmation   
	-2.1719	in             		9.6800	reservation    
	-2.0892	first          		9.4990	invoice        
	-2.0889	rate           		8.2154	trip           
	-2.0590	school         		6.8540	ride           
	-1.9721	thanksgiving   		6.3878	booking        
	-1.8329	enrichment     		6.0268	itinerary      
	-1.6988	user           		5.4079	folio          
	-1.6755	seattle        		5.3746	marriott       
	-1.6117	game           		5.1547	bill           
	-1.6032	rigas          		5.0271	statement      
	-1.5707	2013           		4.2214	ref            
	-1.5569	music          		4.1562	enterprise     
	-1.5550	off            		4.1009	national       
	-1.5222	android        		4.0722	de             
	-1.4898	parse          		3.9322	uber           
	-1.4653	3d             		3.4760	stay           
	-1.4646	stuff   

In [38]:
import eli5
eli5.show_weights(clf, top=10)